In [6]:
#!/usr/bin/python
import numpy as np
import time
import re
def calcuLog(a,b):
    try:
        result = np.log(float(a)/float(b))
    except:
        print "divided by 0"
    return result

def createNgramDict(train_list):
    trainDict = {}
    train_key = ' '.join(train_list[2:])
    train_value = train_list[0]
    return train_key,train_value

#######cannot add trainDict1[train_key] = int(train_value) and return the whole dict because dict is stillbuiding 
####first buiding three dictionaries for 1 gram,bigram and trigram for faster speed ###################
def buildDictionary(input_file):
    try:
        train_data = open(input_file,'r')
    except IOException as err:
        print "no such file"
    trainDict1 = {}
    trainDict2 = {}
    trainDict3 = {}
    for train_sentence in train_data:
        train_list = train_sentence.strip().split(" ")
        if train_list[1] =="1-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict1[train_key] = int(train_value)
        elif train_list[1] == "2-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict2[train_key] = int(train_value)
        elif train_list[1] == "3-GRAM":
            train_key,train_value = createNgramDict(train_list)
            trainDict3[train_key] = int(train_value)
    return trainDict1,trainDict2,trainDict3
#####the q parameter is calculated by trigram divided by bigram########################################

def calQparameter(trigram_each):
    trigram = trigram_each.strip()
    bigram = ' '.join(trigram.split(" ")[0:2])
    try:
        count_tri = trainDict3[trigram]
    except KeyError:
        count_tri = 0
    try:
        count_bi = trainDict2[bigram]
    except KeyError:
        count_bi = 0
    if count_bi != 0:
        result = calcuLog(count_tri,count_bi)
    else:
        result = float('-inf')
    return result
    #print output
    
def cal_emission(count_pair,count_y):
    try:
        emission_para = (float(count_pair)/float(count_y))
    except:
        pass #return "divide by zero"
    return emission_para

#this function finds the tag and the times it appears and save it in a dictionary
def findTagDict(count_file):
    #dictTag = {}#dict()
    try:
        count_content = open(count_file)
    except:
        print "no such file"
    for count_line in count_content:
        count_list = count_line.strip().split(" ")
        if count_list[1] == "1-GRAM":
            if count_list[-1] not in dictTag:
                dictTag[count_list[-1]] = int(count_list[0])
            else:
                dictTag[count_list[-1]] += int(count_list[0])
    #print dictTag
    return dictTag

#this functin finds the word and the frequency of a word tagged aby a specific tag
def findWordDict(count_file):
    dictWord = {}#dict() or will pop dict() not callable error
    try:
        count_content = open(count_file)
    except:
        print "no such file"
    for count_line in count_content:
        count_list = count_line.strip().split(" ")
        word = count_list[-1]
        tag = count_list[2]
        number = count_list[0]
        if count_list[1] =="WORDTAG":
            #initialize
            if word not in dictWord:
                dictWord[word] = {}  
            #add element
            if tag not in dictWord[word]:
                dictWord[word][tag] = int(number)
            else:
                dictWord[word][tag] += int(number)
    for word in dictWord:
        max_ratio = 0
        max_tag = ""
        for tag in dictWord[word]:
            dictWord[word][tag] = np.log(cal_emission(dictWord[word][tag],dictTag[tag]))
    #print dictWord        
    return dictWord

def findEmission(word_input,tag_input):
    pattern1 = r"\b[\d\.\-\"\'\,\?]*\b"
    if word_input.strip() not in dictWord:
        original_word = word_input.strip()
        if original_word.isdigit():
            try:
                value = dictWord["_DIGIT_"][tag_input]
            except:
                value = float("-inf")
        elif original_word.isupper():
            try:
                value = dictWord["_UPPER_"][tag_input]
            except:
                value = float("-inf")
        elif original_word.islower():
            try:
                value = dictWord["_LOWER_"][tag_input]
            except:
                value = float("-inf")
        elif original_word in re.findall(pattern1, original_word):
            try:
                value = dictWord["_NUMERALVALUE_"][tag_input]
            except:
                value = float("-inf")
        else:
            try:
                value = dictWord["_RARE_"][tag_input]
            except:
                value = float("-inf")

    else:#lif test_line.strip() in dictMax://cannot use word and tag because they are the lastelement itmes keys and values
        original_word = word_input.strip()
        try:
            value = dictWord[original_word][tag_input]
        except:
            value = float("-inf")
    return value


           

def Viterbi(sentence):
    #initialization
    n = len(sentence)
    tag_sequence = []
    tag_value = []
    pI = {}
    K_n1 = ['*']
    K_0 = ['*']
    K = ['I-LOC', 'B-ORG', 'I-PER', 'O', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']
    #keys = str(position)","+u+","+v
    pI['0,*,*'] = 0# only store the maximum value corresponding to each position given u and v
    bp = {}
    ###corner case must be taken into consideration or the "0,," in the step 2 and 3 will be generated
    #must consider n==1
    if n == 1:
        optimal_val0 = float("-inf")
        optimal_tag = ''
        for v in K:
            trigram_0 = "*"+" "+"*"+" "+str(v)
            q = calQparameter(trigram_0)
            word_input = sentence[0]
            tag_input = str(v)
            e = findEmission(word_input,tag_input) 
            TEMP = pI['0,*,*']+q+e
            if TEMP > optimal_val0:
                optimal_val0 = TEMP
                optimal_tag = v
        return [optimal_tag],[str(optimal_val0)]
    #########################the first step of Viterbi##################
    for k in range(1,n+1):
        Vk = K
        if k == 1:
            Uk = K_0
            Wk = K_n1    
        elif k == 2:
            Uk = K
            Wk = K_0
        else:
            Uk = K
            Wk = K
        for u in Uk:
            for v in Vk:
                optimal_val = float("-inf")
                optimal_w = ''
                #make u v as given condition,change the value of Wk
                for w in Wk:
                    trigram_each = str(w)+" "+str(u)+" "+str(v)
                    q = calQparameter(trigram_each)
                    word_input = sentence[k-1]
                    tag_input = str(v)
                    e = findEmission(word_input,tag_input) 
                    key_last = str(k-1)+","+str(w)+","+str(u)
                    tmp = float(pI[key_last]) + q + e
                    #print tmp
                    if tmp > optimal_val:
                        optimal_val = tmp
                        optimal_w = w
                    else:
                        optimal_val = optimal_val
                        optimal_w = optimal_w
                key = str(k)+","+str(u)+","+str(v)
                pI[key] = optimal_val
                bp[key] = optimal_w
    
    ##########################step 2 of Viterbi Algorithm######################
    max_u = ""
    max_v = ""
    max_value = float("-inf")
    for u in K:
        for v in K:
            trigram_each = str(u)+" "+str(v)+" "+"STOP"
            q = calQparameter(trigram_each)
            key_fin = str(n)+","+str(u)+","+str(v)
            temp = pI[key_fin] + q
            if temp > max_value:
                max_value = temp
                max_u = u
                max_v = v
    #print pI
    #print bp
    tag_sequence.insert(0,max_v)
    tag_sequence.insert(0,max_u)
    ######################step 3 of Viterbi Algorithm#########################
    #insert all y sequence backwards
    #insert n~2 sequence of pi values backwards
    for k in range(n-2,0,-1):
        key_bp = str(k+2)+","+str(tag_sequence[0])+","+str(tag_sequence[1])
        #print key_bp
        tag_value.insert(0,str(pI[key_bp]))
        y_k = bp[key_bp]
        #print y_k
        tag_sequence.insert(0,y_k)
        #print len(tag_sequence)
    #insert the second value of pi
    key1 = str(2)+","+str(tag_sequence[0])+","+str(tag_sequence[1])
    tag_value.insert(0,str(pI[key1]))
    #insert the first value of pi
    key0 = str(1)+","+"*"+","+str(tag_sequence[0])
    tag_value.insert(0,str(pI[key0]))
    return tag_sequence,tag_value
                

def testFile(test_file):
    word_list = []
    sentence_list = []
    try:
        testdata = open(test_file,'r')
    except:
        print "no such file"
    for each_word_raw in testdata:
        if each_word_raw != "\n":
            each_word = each_word_raw.strip()
            word_list.append(each_word)
        else:
            sentence_list.append(word_list)
            word_list = []
    return sentence_list
        

if __name__ == "__main__":
    input_file = "ner_rare_multi.counts"  
    test_file = "ner_dev.dat"
    t_start = time.time()
    trainDict1,trainDict2,trainDict3 = buildDictionary(input_file)
    dictTag = {}
    dictTag = findTagDict(input_file)
    dictWord = {}
    dictWord = findWordDict(input_file)
    sentence_list = testFile(test_file)
    outputfile = open("6.txt","w")
    for word_list in sentence_list:
        tag_sequence,tag_value = Viterbi(word_list)
        tuple_sentence = zip(word_list,tag_sequence,tag_value)
        for tuple_word in tuple_sentence:
            output = " ".join(tuple_word)+"\n"
            outputfile.write(output)
            print output
        outputfile.write("\n")
        print "\n"

/Users/yt2583columbia.edu/anaconda3/envs/Python27/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


CRICKET O -8.67251828825213

- O -13.873617328135193

LEICESTERSHIRE I-ORG -20.84514536811389

TAKE I-ORG -24.478374802123668

OVER I-ORG -28.351308241238122

AT O -35.95112377441183

TOP I-ORG -40.68077858629462

AFTER O -50.73226165068148

INNINGS I-ORG -55.46191646256427

VICTORY I-ORG -59.095145896574046

. O -62.666071839009945



LONDON I-LOC -6.673647192389261

1996-08-30 O -16.674703391247146



West I-MISC -9.809426955934672

Indian I-MISC -16.332531147236224

all-rounder O -19.410942649347298

Phil I-PER -30.053269913884808

Simmons I-PER -31.19172824980498

took O -38.99965069238994

four O -46.82308771113671

for O -51.77582803705826

38 O -61.08670067745426

on O -65.66920269807602

Friday O -72.67195006510237

as O -78.47293701808572

Leicestershire I-ORG -89.69612987487506

beat O -96.49843176489838

Somerset I-ORG -105.73106582187272

by O -111.77381807676046

an O -118.03425052715225

innings O -125.91800681190811

and O -130.18581282257279

39 O -139.74799989124972

r

Derbyshire I-ORG -3.105361500498466

kept O -12.39519784273813

up O -19.13474831081286

the O -22.45124398265385

hunt O -32.7065782318907

for O -37.659318557812256

their O -44.0141241810467

first O -50.25436323971559

championship O -58.60546003629769

title O -67.51086756858552

since O -74.82176283865594

1936 O -80.22283716864591

by O -85.85040139997743

reducing O -88.61544556748497

Worcestershire I-ORG -99.83863842427431

to O -104.32884831666418

133 O -110.01762170234495

for O -114.9703620282665

five O -122.45310755526359

in O -126.52576675318126

their O -132.8805723764157

second O -139.7065383670237

innings O -147.59029465177957

, O -150.89933995803207

still O -158.66976755748095

100 O -166.60833207873182

runs O -174.7491334664776

away O -183.2127082464864

from O -188.78458939134111

avoiding O -191.54963355884868

an O -197.5223669535497

innings O -205.40612323830555

defeat O -214.51632518323942

. O -217.81348266759264



After O -9.365665468812075

the O

Bristol I-ORG -8.757070649526318

: O -14.733448524411834

Gloucestershire I-ORG -23.514097457643103

183 O -29.3303598111893

and O -33.88586487754479

185-6 O -38.21694332936711

( O -42.46451794900948

J. I-PER -52.73034074139494

Russell I-PER -59.608970204324876

56 O -68.45178912119184

not O -74.75326591366556

out O -81.28118704478624

) O -85.5287616644286

, O -88.83780697068107

Northamptonshire I-ORG -100.1787828631268

190 O -105.99504521667299

( O -110.53031889200616

K. I-PER -121.68344487939254

Curran I-PER -122.82190321531272

52 O -131.82506478225486

; O -140.45943201721795

A. I-PER -150.09273225085988

Smith I-PER -157.7251335161662

5-68 O -161.99362335256765

) O -166.6255645274624

. O -169.9227220118156



CRICKET O -8.67251828825213

- O -13.873617328135193

1997 O -22.337192108143995

ASHES I-ORG -29.30872014812269

INTINERARY I-ORG -32.941949582132466

. O -36.512875524568365



LONDON I-LOC -6.673647192389261

1996-08-30 O -16.674703391247146



Australia


24-28 O -12.445637628469616

Fourth O -16.98912675562692

test O -25.429171038225526

( O -29.676745657867897

at O -35.042805470012766

Headingley I-LOC -46.182056327666054

) O -50.460015545201145



August O -8.009830315176893



August O -8.009830315176893

1-4 O -12.466046500100823

v O -20.178654528660854

Somerset I-ORG -31.653161813731092

( O -36.31592445692969

four O -44.04269397611492

days O -51.939873281202914

) O -56.187447900845285



August O -8.009830315176893

7-11 O -12.466046500100823

Fifth O -17.009535627258128

test O -25.449579909856737

( O -29.69715452949911

at O -35.06321434164398

Trent I-LOC -46.202465199297265

Bridge I-LOC -55.03400765266341

, O -58.34498876665371

Nottingham I-LOC -68.70190773816402

) O -72.97986695569911



August O -8.009830315176893

16-18 O -12.466046500100823

v O -20.178654528660854

Kent I-ORG -31.51963042110657

( O -36.18239306430517

three O -43.368031610229075

days O -51.26521091531707

) O -55.51278553495944



August 

...

The O -5.587859805503737

ministry O -14.303324024253287

spokesman O -21.84112932843339

said O -26.521597311662763

the O -29.838092983503753

German I-MISC -37.88177573034342

embassy O -46.696233221727155

immediately O -55.50853213800568

made O -63.05581618614032

a O -67.25269501945291

sharp O -77.50802926868977

protest O -86.61823121362362

to O -90.69325308245726

the O -94.00974875429824

Tehran I-LOC -105.14899961195152

government O -111.8276690232148

. O -115.46903925032407



The O -5.587859805503737

Iranian I-MISC -14.547384206551234

ambassador O -24.432283109636384

was O -29.64726976532646

also O -36.48253814859678

summoned O -39.24758231610433

to O -43.322604184937965

the O -46.63909985677895

ministry O -55.22942634242689

in O -59.302085540344564

Bonn I-LOC -70.15365432554607

to O -74.25906079227244

hear O -84.85860778426536

a O -89.05548661757796

sharp O -99.31082086681481

protest O -108.42102281174866

and O -112.68882882241333

" O -117.20391091721

Nyerere I-ORG -3.105361500498466

arrived O -12.39519784273813

in O -16.755556096346616

Rome I-LOC -27.607124881548124

this O -34.1897796395718

week O -41.74480419384131

on O -46.327306214463064

a O -50.52418504777566

private O -59.32423206440567

visit O -67.30596875752173

and O -71.5737747681864

held O -79.39299253180468

talks O -87.17539632230024

with O -92.64323882875505

the O -95.95973450059603

U.S. I-LOC -103.15740355057962

special O -112.61644382452472

envoy O -123.08245942389313

to O -127.15748129272677

Burundi I-LOC -137.31590289736832

, O -140.65533280151354

Howard I-PER -151.92164832254534

Wolpe I-PER -153.06010665846551

, O -156.30656334929714

and O -160.95873591521416

the O -164.27523158705517

Sant I-ORG -169.84671529481665

' O -177.5879503934639

Egidio I-ORG -180.91756087312947

Community I-ORG -189.901067412618

, O -193.4838811769532

an O -199.744313627345

Italian I-MISC -208.66346511153859

Roman B-MISC -217.05455638615717

Catholic I-MISC -

A O -7.296274262985741

policeman O -10.186456163594894

in O -14.259115361512567

the O -17.575611033353557

southern O -25.94821403515663

Belgian I-MISC -35.09050907066441

town O -43.382777179631844

of O -47.40287324562043

Chatelet I-ORG -52.97435695338191

told O -60.29748455050581

Reuters I-ORG -67.10658090406844

that O -73.08052652454957

thieves O -76.13326974774792

used O -84.90052694155494

a O -89.09740577486754

car O -98.35421119399327

to O -102.42923306282691

ram O -105.19427723033445

the O -108.51077290217543

window O -119.1025793880335

of O -123.11042355437357

an O -129.08315694907455

arms O -138.3399623682003

store O -148.59529661743716

in O -152.66795581535484

neighbouring O -162.47130494084865

Chatelineaux O -167.01479406800598

last O -173.57413139536004

night O -181.99118615973396

. O -185.28834364408718



It O -7.2518225004149075

was O -12.57969498955808

the O -15.896190661399068

second O -22.722156652007072

arms O -31.978962071132802

robbe

Jordan I-LOC -8.028192855194572

defends O -18.617035718954575

his O -24.87463429984103

decision O -33.31467858243964

to O -37.38970045127328

make O -45.5478935817309

the O -48.86438925357188

film O -58.76304855887

, O -62.07209386512247

whose O -71.13784404748549

screenplay O -73.90288821499303

he O -79.73766216555376

wrote O -89.63632147085188

himself O -98.65951203879611

after O -104.70589834307185

years O -112.00930294141311

of O -116.01714710775317

research O -125.9158064130513

, O -129.22485171930379

saying O -137.1220310243918

it O -143.00791742116164

was O -148.2106521772032

" O -152.72573427200248

more O -159.89002606788102

about O -166.898313615283

history O -176.61465136378717

than O -183.84608246250335

any O -191.5255382837465

political O -199.46410280499737

statement O -207.01138685313202

" O -211.5264689479313

. O -214.82362643228453



" O -4.89837899018737

The O -10.228079633404604

film O -20.12673893870273

spares O -22.89178310621028

n

" O -4.89837899018737

I O -11.497913373993947

have O -17.482551671201254

made O -25.0298357193359

choices O -27.79487988684345

about O -34.80316743424541

certain O -44.701826739543534

events O -55.11131166860765

based O -64.09368024203161

on O -68.67618226265336

my O -76.96540365551738

own O -85.52906189408316

extensive O -88.2941060615907

research O -98.19276536688882

into O -105.31883594994716

his O -111.23222178807754

letters O -121.82402827393561

and O -126.09183428460028

reported O -133.7822191763756

speeches O -136.54726334388317

, O -139.85630865013567

" O -144.37139074493496

he O -150.2061646954957

said O -154.88663267872508

. O -158.1837901630783

" O -162.69887225787758



I O -6.8576935460930795

wanted O -16.239636698320414

to O -20.31465856715406

make O -28.472851697611677

this O -35.02512185774264

a O -39.22200069105523

story O -41.987044858562776

as O -47.78803181154612

accurate O -50.553075979053666

as O -56.35406293203701

possible O -65